In [ ]:
"""
fred_regression_example.py

A simple script to fetch FRED data (FEDFUNDS and CPI) from the FRED API,
perform a linear regression of CPI on FEDFUNDS, and plot the results.

Requirements:
- pandas
- pandas_datareader
- statsmodels
- matplotlib

Usage:
1. Obtain a FRED API key from https://fred.stlouisfed.org/
2. Set environment variable in your terminal:
   set FRED_API_KEY=YOUR_KEY  # Windows CMD
   $Env:FRED_API_KEY='YOUR_KEY'  # PowerShell
3. Run the script:
   python fred_regression_example.py

Outputs:
- regression_summary.txt: Model summary statistics
- cpi_vs_fedfunds.png: Scatter plot with regression line
"""

In [ ]:
import os
import datetime
import pandas as pd
import matplotlib.pyplot as plt
from pandas_datareader import data as web
import statsmodels.api as sm

In [ ]:
def fetch_fred_series(series_id, start, end, api_key):
    return web.DataReader(series_id, "fred", start, end, api_key=api_key)

In [ ]:
def main():
    api_key = os.getenv("FRED_API_KEY")
    if not api_key:
        raise RuntimeError("Please set FRED_API_KEY environment variable")

    # Define date range
    start = datetime.datetime(2000, 1, 1)
    end = datetime.datetime.today()

    # Fetch data
    fedfunds = fetch_fred_series("FEDFUNDS", start, end, api_key)
    cpi = fetch_fred_series("CPIAUCSL", start, end, api_key)

    # Combine and clean
    df = pd.concat([fedfunds, cpi], axis=1).dropna()
    df.columns = ["FEDFUNDS", "CPI"]

    # Regression
    X = sm.add_constant(df["FEDFUNDS"])
    model = sm.OLS(df["CPI"], X).fit()

    # Save summary to text file
    with open("regression_summary.txt", "w") as f:
        f.write(model.summary().as_text())

    # Print summary to console
    print(model.summary())

    # Plot
    plt.figure()
    plt.scatter(df["FEDFUNDS"], df["CPI"], alpha=0.5)
    plt.plot(df["FEDFUNDS"], model.predict(X), label="Fit Line")
    plt.title("CPI vs FEDFUNDS (2000-present)")
    plt.xlabel("FEDFUNDS (%)")
    plt.ylabel("CPI Index")
    plt.legend()
    plt.savefig("cpi_vs_fedfunds.png", dpi=300)
    plt.show()

In [ ]:
if __name__ == "__main__":
    main()